# loop on the OCs to estimate isochrone and mass

In [1]:
using PyCall

## PopStar modules...
synthetic= pyimport("popstar.synthetic")
evolution= pyimport("popstar.evolution")
atmospheres= pyimport("popstar.atmospheres")
reddening= pyimport("popstar.reddening")

using Distributions, Statistics, Distances
using DataFrames
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = ENV["GAIA_ROOT"]

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"
isodir= "$wdir/isochrones"

cd(wdir)

In [2]:
### functions

function mass_isochrone(f)
    
    fname= f[1:end-4] ; fvot= f[1:end-7] 
    ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
    ocmassfile= "$ocdir/$fname-mass.csv"
    
    df= CSV.read(ocfile , delim= ";")
    
    t= split(fvot,"-")
    if length(t) == 3
        t[1]= t[1]*"-"*t[2]
    end
    ocname= t[1]
    
    println("## Name: $fname")
    logage= df1[(df1.name .== ocname), :log_age]
    # println(logage[1])
    
    ################################
    ## ISOCHRONES ##################
    ################################
    agmedian= median(df.ag[.! isnan.(df.ag)])
    distance= median(df.distance)
    nstar= length(df.distance)
    
    println("## N(star): $nstar")
    # Define isochrone parameters
    logAge= logage[1]  # Age in log(years)
    AKs= 0.0 # extinction in mags
    dist= distance # distance in parsec
    metallicity= 0.0 # Metallicity in [M/H]
    
    println("### log Age: $logAge")
    println("### AKs: $AKs")
    println("### distance: $dist")
    println("### metallicity: $metallicity")
    
    ## Define evolution/atmosphere models and extinction law
    if logAge>6.5
        evo_model = evolution.MergedBaraffePisaEkstromParsec() 
    else
        evo_model= evolution.MISTv1()
    end
    
    atm_func = atmospheres.get_merged_atmosphere
    red_law = reddening.RedLawHosek18b()

    ## Filter
    filt_list = [ "gaia,dr2_rev,Gbp", "gaia,dr2_rev,G"  ,"gaia,dr2_rev,Grp"]
    
    #####################################
    println("### Computing isochrones..")
    iso = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list, iso_dir= isodir)
    
    println("### Updating the DF and creating the new one with stellar mass..")
    
    data= iso.points
    println(data.keys())
    ndata= size(data)[1]

    mass= zeros(ndata)
    Gmag= zeros(ndata)
    BPmag= zeros(ndata)
    RPmag= zeros(ndata)
    color= zeros(ndata)

    for i in 1:ndata
        mass[i]= data[i][4]
        Gmag[i]= data[i][10]
        BPmag[i]= data[i][9]
        RPmag[i]= data[i][11]
        color[i]= BPmag[i]-RPmag[i]
    end

    ## Cluster
    BmR= df.bp .- df.rp
    GMAG= df.gbar + 5 .* log10.(df.distance) .- 17.
    

    ### Stellar mass 
    ################
    nxy= length(mass)
    A= Array{Float64}(undef,2,nxy)
    for i in 1:nxy
        A[1,i]= color[i]
        A[2,i]= Gmag[i]
    end
    
    d= Distances.Euclidean()
    mstar= zeros(length(BmR))

    for i in 1:length(BmR)
        starm= [BmR[i]; GMAG[i]]
        r= Distances.colwise(d, A, starm)
    
        idx= argmin(r)
        mstar[i]= mass[idx]
    end
    
    df.mass= mstar
    
    ## Some Logs mass stats...
    massmean= mean(mstar)
    println("### Mass (mean): $massmean")
    
    ### save the OC df
    CSV.write(ocmassfile, df, delim=";") 
    println("\n\n")    
end

mass_isochrone (generic function with 1 method)

In [3]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

sample= rootdir*"/master/notebooks/data/BrowseTargets.18292.1530479692.fake"
fileres= "isolist-done.csv"

df1= CSV.read(sample, delim= "|")
df1.name= strip.(df1.name)

noc= 0
for f in files
    filered= CSV.read(fileres,delim=";",copycols= true)
    
    if !(f in filered.fileoc)
        println(f)
        mass_isochrone(f)
        push!(filered,(f,))
        CSV.write(fileres,filered,delim=";")
    end
    
end

NGC 2925-1.4deg-oc.csv
## Name: NGC 2925-1.4deg-oc
## N(star): 146
### log Age: 7.805
### AKs: 0.0
### distance: 773.1143276028146
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]


┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:72
└ @ Main ./In[2]:72
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:72
└ @ Main ./In[2]:72
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:73
└ @ Main ./In[2]:73
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:73
└ @ Main ./In[2]:73
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:74
└ @ Main ./In[2]:74
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:74
└ @ Main ./In[2]:74
┌ Warning: `getindex(o::PyObject, 

### Mass (mean): 0.9582873561643838



NGC 3114-1.7deg-oc.csv
## Name: NGC 3114-1.7deg-oc
## N(star): 777
### log Age: 8.3
### AKs: 0.0
### distance: 1045.3686288124686
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.2686664903474902



NGC 3228-1.0deg-oc.csv
## Name: NGC 3228-1.0deg-oc
## N(star): 62
### log Age: 8.42
### AKs: 0.0
### distance: 489.01467865739755
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.208757983870968



NGC 3532-2.1deg-oc.csv
## Name: NGC 3532-2.1deg-oc
## N(star): 1874
### log Age: 8.65
### AKs: 0.0
### distance: 483.91318971828287
### metallicity: 0.0
### Computing isochrones..
### Upda

Changing to logg=2.50 for T= 16144 logg=2.30### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.8030999999999999



NGC 3680-1.0deg-oc.csv
## Name: NGC 3680-1.0deg-oc
## N(star): 71
### log Age: 9.2
### AKs: 0.0
### distance: 1088.8310667056649
### metallicity: 0.0
### Computing isochrones..

Changing to logg=2.50 for T= 16444 logg=2.33
Changing to logg=2.50 for T= 16796 logg=2.37
Changing to logg=2.50 for T= 17183 logg=2.41
Changing to logg=2.50 for T= 17575 logg=2.45
Changing to logg=2.50 for T= 17795 logg=2.47
Changing to logg=2.50 for T= 17848 logg=2.48
Changing to logg=2.50 for T= 17902 logg=2.48
Changing to logg=2.50 for T= 17951 logg=2.49
Changing to logg=2.50 for T= 18009 logg=2.49
Changing to logg=2.50 for T= 18063 logg=2.50
Changing to logg=3.00 for T= 19015 logg=2.58
Changing to logg=3.00 for T= 19090 logg=2.59
Changing to logg=3.00 for T= 

Starting synthetic photometry### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.2515210704225355



NGC 381-1.0deg-oc.csv
## Name: NGC 381-1.0deg-oc
## N(star): 124
### log Age: 8.85
### AKs: 0.0
### distance: 1175.6518449868693
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.1181250241935479



NGC 4103-1.1deg-oc.csv
## Name: NGC 4103-1.1deg-oc
## N(star): 232
### log Age: 7.755
### AKs: 0.0
### distance: 1029.1086536552154
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.1859394

...

Changing to logg=2.50 for T= 11825 logg=2.49### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.2160235714285716



Platais 3-11.2deg-oc.csv
## Name: Platais 3-11.2deg-oc
## N(star): 106
### log Age: 8.8
### AKs: 0.0
### distance: 179.45535069260447
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.6500504905660379



Platais 4-3.3deg-oc.csv
## Name: Platais 4-3.3deg-oc
## N(star): 211
### log Age: 8.55
### AKs: 0.0
### distance: 732.4279613059558
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Ma

M =   0.070 Msun  T =  2807 K  m_hst_Grp = 22.48### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.9679337243401758



Platais 8-6.0deg-oc.csv
## Name: Platais 8-6.0deg-oc
## N(star): 396
### log Age: 7.75
### AKs: 0.0
### distance: 136.1742107777265
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.40312601010101



Platais 9-5.5deg-oc.csv
## Name: Platais 9-5.5deg-oc
## N(star): 374
### log Age: 8.09
### AKs: 0.0
### distance: 438.55254099047727
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### M

Changing to logg=5.00 for T= 66697 logg=4.24### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.4845994424149816



Roslund 5-2.1deg-oc.csv
## Name: Roslund 5-2.1deg-oc
## N(star): 115
### log Age: 7.57
### AKs: 0.0
### distance: 544.1837509228311
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.9719391304347825



Ruprecht 111-1.0deg-oc.csv
## Name: Ruprecht 111-1.0deg-oc
## N(star): 53
### log Age: 8.92
### AKs: 0.0
### distance: 1502.1811386402374
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
##

M =   0.090 Msun  T =  2315 K  m_hst_Grp = 23.80### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0514371111111112



Ruprecht 147-6.7deg-oc.csv
## Name: Ruprecht 147-6.7deg-oc
## N(star): 124
### log Age: 9.33
### AKs: 0.0
### distance: 307.44733560561787
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0693386612903226



Ruprecht 150-1.0deg-oc.csv
## Name: Ruprecht 150-1.0deg-oc
## N(star): 137
### log Age: 8.7
### AKs: 0.0
### distance: 849.1438380325056
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hs

     Starting at: ### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.8813361548117157



Ruprecht 65-1.0deg-oc.csv
## Name: Ruprecht 65-1.0deg-oc
## N(star): 84
### log Age: 8.82
### AKs: 0.0
### distance: 706.0259817622582
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0556164880952381



Ruprecht 67-1.0deg-oc.csv
## Name: Ruprecht 67-1.0deg-oc
## N(star): 61
### log Age: 8.6
### AKs: 0.0
### distance: 435.9524683570372
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.7552247213